In [13]:
import requests
import pandas as pd
import ivolatility as ivol
import matplotlib.pyplot as plt
import numpy as np
import holidays
# ivolatility connection
apiKey = 'fFF8EEL54op7k0DN'
ivol.setLoginParams(apiKey=apiKey)

In [14]:
# Variables
symbol = 'AAPL'
date = '2024-04-30'

In [15]:
# Get stock EOD Price
def fetch_eod_stock_prices(api_key, symbol, date):
    url = f"https://restapi.ivolatility.com/equities/eod/stock-prices?apiKey={api_key}&symbol={symbol}&date={date}"
    response = requests.get(url)
    data = response.json()
    # Assuming the relevant data is under the 'data' key
    df = pd.DataFrame(data['data'])
    return df
Current = fetch_eod_stock_prices('fFF8EEL54op7k0DN', symbol, date)
Current

,record_no,date,symbol,exchange,open,high,low,close,volume,unadjusted_close
0,1,2024-04-30,AAPL,NASDAQ,173.33,174.99,170.0,170.33,65321081,170.33


In [4]:
getMarketData = ivol.setMethod('/equities/eod/hv')
marketData = getMarketData(symbol=symbol, date=date)
df_H = pd.DataFrame(marketData)
df_H

,record_no,symbol,exchange,date,price,10d HV,20d HV,30d HV,60d HV,90d HV,120d HV,150d HV,180d HV
0,1,AAPL,NASDAQ,2024-04-30,170.33,0.201656,0.243554,0.253854,0.210647,0.208596,0.196592,0.193689,0.201202


In [5]:
# Get Nearest Option Price
getMarketData = ivol.setMethod('/equities/eod/nearest-option-tickers')
marketData = getMarketData(symbol=symbol, startingDate=date, dte=60, moneyness=0, callPut='P')
df_near = pd.DataFrame(marketData)
df_near

,option_id,option_symbol,comment
0,119715046,AAPL 240621P00170000,NaN
1,119715048,AAPL 240621P00175000,NaN


In [6]:
first_option_id = df_near.iloc[0]["option_id"]
first_option_symbol= df_near.iloc[0]["option_symbol"]
print(first_option_symbol)

AAPL  240621P00170000


In [29]:
def create_working_days(start_date, end_date, country='US'):
    # Generate a date range for all days between start and end
    all_days = pd.date_range(start=start_date, end=end_date, freq='D')
    
    # Fetch holidays for the given country in the range of years
    us_holidays = holidays.CountryHoliday(country, years=[start_date.year, end_date.year])
    
    # Filter out weekends and holidays, and format dates as strings
    working_days = [day.strftime('%Y-%m-%d') for day in all_days if day.weekday() < 5 and day not in us_holidays]
    
    return working_days

# Example usage: create a list of working days from January 1, 2024, to January 31, 2024
date_range = create_working_days(pd.Timestamp('2024-04-04'), pd.Timestamp('2024-04-05'))
print(date_range)


['2024-04-04', '2024-04-05']


In [30]:
getMarketData = ivol.setMethod('/equities/intraday/single-equity-optionsymbol-rawiv')
#marketData = getMarketData(optionSymbol=first_option_symbol, date=date, minuteType='HOUR')
marketData = getMarketData(optionSymbol=first_option_symbol, date=date, minuteType='HOUR')

df_IV = pd.DataFrame(marketData)
last_index_value = df_IV.index[-1]
ImpVol = df_IV.iloc[last_index_value]["optionIv"]

df_IV.tail(1)

400 Limit is exceeded


IndexError: index -1 is out of bounds for axis 0 with size 0

In [31]:
print('La Volatilidad Implícita es:', round(ImpVol,3)*100)
HistVol = df_H.iloc[0]["180d HV"]
print('La Volatilidad Histórica es:', round(HistVol,3)*100)
# Normalización IVOL
ratio = ImpVol/HistVol *100
ratio = round(ratio,1)
print('El Ratio es:', ratio,'%')

NameError: name 'ImpVol' is not defined

In [32]:
def get_IV(first_option_symbol, date):
    """
    Obtiene la volatilidad implícita para una opción específica en una fecha dada, filtrando por columnas específicas.

    Parámetros:
    first_option_symbol : símbolo de la opción para la cual obtener los datos.
    date : fecha para la cual obtener los datos de volatilidad implícita.

    Retorna:
    Un DataFrame de pandas con la última entrada de la volatilidad implícita, incluyendo solo las columnas especificadas.
    """
    # Suponiendo que 'ivol' es un objeto global o está definido previamente
    # Configurar el método de obtención de datos del mercado
    getMarketData = ivol.setMethod('/equities/intraday/single-equity-optionsymbol-rawiv')
    # Obtener los datos del mercado para el símbolo y fecha especificados
    marketData = getMarketData(optionSymbol=first_option_symbol, date=date, minuteType='HOUR')
    # Convertir los datos del mercado en un DataFrame
    df_IV = pd.DataFrame(marketData)
    # Filtrar las columnas deseadas
    columns = ['timestamp', 'stockSymbol', 'underlyingPrice', 'optionIv', 'optionDelta', 'optionGamma', 'optionTheta', 'optionVega']
    df_IV = df_IV[columns]
    # Devolver la última fila del DataFrame
    return df_IV.tail(1)

In [36]:
#date_range = pd.date_range(start='2024-04-23', end='2024-04-24')
date_range = ['2024-04-04', '2024-04-05']

# Iterar sobre el rango de fechas y obtener la volatilidad implícita para cada fecha
results = []
for date in date_range:
    iv_data = get_IV(first_option_symbol=first_option_symbol, date=date)
    if not iv_data.empty:
        results.append(iv_data)
    else:
        print(f"No market data for {date.strftime('%Y-%m-%d')}")

# Concatenar todos los DataFrames en uno solo si hay resultados
if results:
    df = pd.concat(results, ignore_index=True)
    df['HistVol'] = HistVol  # Asumimos que HistVol es una constante o está calculada en otro lugar
    df['OptionHV'] = round(df['HistVol'], 3)
    df['IV_HV_Ratio'] = (df['optionIv'] / df['OptionHV']) * 100
    df = df.round(2)
else:
    df = pd.DataFrame()  # Crea un DataFrame vacío si no hay datos de mercado

df

400 Limit is exceeded


KeyError: "None of [Index(['timestamp', 'stockSymbol', 'underlyingPrice', 'optionIv',\n       'optionDelta', 'optionGamma', 'optionTheta', 'optionVega'],\n      dtype='object')] are in the [columns]"

In [ ]:
date_range = pd.date_range(start='2024-04-24', end='2024-04-27')

# Iterate over the date range and get the implied volatility for each date
results = []
for date in date_range:
    iv_data = get_IV(first_option_symbol=first_option_symbol, date=date.strftime('%Y-%m-%d'))
    if not iv_data.empty:
        results.append(iv_data)
    else:
        print(f"No market data for {date.strftime('%Y-%m-%d')} due to a non-trading day or missing data.")

# Concatenate all DataFrames into one if results are available
if results:
    df = pd.concat(results, ignore_index=True)
    # Assuming HistVol needs to be calculated or assigned, here is a placeholder
    df['HistVol'] = 0.25  # This is a hypothetical value
    df['OptionHV'] = round(df['HistVol'], 3)
    df['IV_HV_Ratio'] = (df['optionIv'] / df['OptionHV']) * 100
    df = df.round(2)
else:
    print('Sin datos')

df

In [ ]:
# Rango de fechas desde '2024-04-20' hasta '2024-05-02'
date_range = pd.date_range(start='2024-04-22', end='2024-04-24')

# Iterar sobre el rango de fechas y obtener la volatilidad implícita para cada fecha
results = []
for date in date_range:
    iv_data = get_IV(first_option_symbol='AAPL', date=date.strftime('%Y-%m-%d'))
    if not iv_data.empty:
        results.append(iv_data)
    else:
        print(f"No market data for {date.strftime('%Y-%m-%d')} due to a non-trading day or missing data.")

# Concatenar todos los DataFrames en uno solo si hay resultados
if results:
    df = pd.concat(results, ignore_index=True)
    df['HistVol'] = HistVol  # Asumimos que HistVol es una constante o está calculada en otro lugar
    df['OptionHV'] = round(df['HistVol'], 3)
    df['IV_HV_Ratio'] = (df['optionIv'] / df['OptionHV']) * 100
    df = df.round(2)
else:
    df = pd.DataFrame()  # Crea un DataFrame vacío si no hay datos de mercado

df

In [ ]:
date_range = pd.date_range(start='2024-04-25', end='2024-04-27')

# Iterar sobre el rango de fechas y obtener la volatilidad implícita para cada fecha
results = []
for date in date_range:
    iv_data = get_IV(first_option_symbol=first_option_symbol, date=date.strftime('%Y-%m-%d'))
    if not iv_data.empty:
        results.append(iv_data)
    else:
        print(f"No market data for {date.strftime('%Y-%m-%d')}")

# Concatenar todos los DataFrames en uno solo si hay resultados
if results:
    df = pd.concat(results, ignore_index=True)
    df['HistVol'] = HistVol  # Asumimos que HistVol es una constante o está calculada en otro lugar
    df['OptionHV'] = round(df['HistVol'], 3)
    df['IV_HV_Ratio'] = (df['optionIv'] / df['OptionHV']) * 100
    df = df.round(2)
else:
    df = pd.DataFrame()  # Crea un DataFrame vacío si no hay datos de mercado

df

In [ ]:
# Rango de fechas desde '2024-04-20' hasta '2024-05-02'
date_range = pd.date_range(start='2024-04-22', end='2024-04-24')

# Iterar sobre el rango de fechas y obtener la volatilidad implícita para cada fecha
results = []
for date in date_range:
    iv_data = get_IV(first_option_symbol=first_option_symbol, date=date.strftime('%Y-%m-%d'))
    results.append(iv_data)

# Concatenar todos los DataFrames en uno solo
df = pd.concat(results, ignore_index=True)

In [ ]:
df['HistVol'] = HistVol 
df['OptionHV'] = round(df['HistVol'], 3)
df['IV_HV_Ratio'] = (df['optionIv'] / df['OptionHV']) * 100
df = df.round(2)
df